<a href="https://colab.research.google.com/github/88FaHaD/ML/blob/main/birdclassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf


In [2]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 4084, done.
remote: Counting objects: 100% (4084/4084), done.
remote: Compressing objects: 100% (3075/3075), done.
remote: Total 4084 (delta 1189), reused 2902 (delta 949), pack-reused 0
Receiving objects: 100% (4084/4084), 44.61 MiB | 23.89 MiB/s, done.
Resolving deltas: 100% (1189/1189), done.


In [3]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━

In [4]:
#run model builder test
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

2024-03-06 08:25:20.936295: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-06 08:25:20.936412: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-06 08:25:20.938083: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-06 08:25:22.777166: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Running tests under Python 3.10.12: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
W0306 08:25:29.364954 132216021889024 batch_normalization.py:1531] `tf.keras.layers.experimental.SyncBatchNormalization` end

In [5]:
import pandas as pd

In [6]:
! pip install kaggle

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
! mkdir ~/.kaggle

In [9]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [10]:
! chmod 600 ~/.kaggle/kaggle.json

In [11]:
!kaggle datasets download gpiosenka/100-bird-species

 99% 1.94G/1.96G [00:22<00:00, 150MB/s]
100% 1.96G/1.96G [00:22<00:00, 92.3MB/s]


In [12]:
import zipfile

# Specify the path to the zip file
zip_file_path = "/content/100-bird-species.zip"

# Specify the directory where you want to extract the contents
extract_to_directory = "/content/bird_species_data"

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_directory)



In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (160, 160)
BATCH_SIZE = 32

train_dir = '/content/bird_species_data/train'
valid_dir = '/content/bird_species_data/valid'
test_dir=   '/content/bird_species_data/test'

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Training images:")
train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=IMAGE_SHAPE,
                                               batch_size=BATCH_SIZE,
                                               class_mode="categorical")

print("validdation images:")
valid_data = train_datagen.flow_from_directory(valid_dir,
                                              target_size=IMAGE_SHAPE,
                                              batch_size=BATCH_SIZE,
                                              class_mode="categorical")



Training images:
Found 84635 images belonging to 525 classes.
validdation images:
Found 2625 images belonging to 525 classes.


In [14]:
# Create tensorboard callback (functionized because need to create a new one for each model)
import datetime
def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

In [15]:
import tensorflow_hub as hub
from tensorflow.keras import layers

In [16]:
mobilenet="https://www.kaggle.com/models/google/mobilenet-v2/frameworks/TensorFlow2/variations/035-160-feature-vector/versions/2"
effinet="https://www.kaggle.com/models/google/efficientnet-v2/frameworks/TensorFlow2/variations/imagenet1k-b1-feature-vector/versions/2"


In [17]:
def create_model(model_url,num_classes=10):
  #downloading the pretrained model and save it as keras layer
  feature_extraction_layer=hub.KerasLayer(model_url,
                                          trainable=False, # freezing already learned patterns by model
                                          name='Feature_extraction_layer',
                                          input_shape=IMAGE_SHAPE +(3,))
  #The model will pass through pretrained model  and the then will move to our model
  model=tf.keras.Sequential([
      feature_extraction_layer,
      layers.Dense(num_classes,activation='softmax',name='oytput_layer')
  ])

  return model

In [18]:
mobilenet_model=create_model(mobilenet,num_classes=len(os.listdir('/content/bird_species_data/train')))

In [19]:
mobilenet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Feature_extraction_layer (  (None, 1280)              410208    
 KerasLayer)                                                     
                                                                 
 oytput_layer (Dense)        (None, 525)               672525    
                                                                 
Total params: 1082733 (4.13 MB)
Trainable params: 672525 (2.57 MB)
Non-trainable params: 410208 (1.56 MB)
_________________________________________________________________


In [20]:
mobilenet_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                        metrics=['accuracy']
)

In [ ]:
mobile_history=mobilenet_model.fit(train_data,
                                   epochs=10,
                                   steps_per_epoch=(len(train_data)),
                                   validation_data=valid_data,
                                   validation_steps=len(valid_data),
                                   callbacks=[create_tensorboard_callback(dir_name='tensorflow_hub',
                                                                          experiment_name='mobilenetv2')]
                                   )


Saving TensorBoard log files to: tensorflow_hub/mobilenetv2/20240306-082830
Epoch 1/10
2645/2645 [==============================] - 752s 281ms/step - loss: 1.4988 - accuracy: 0.6685 - val_loss: 0.6841 - val_accuracy: 0.8263
Epoch 2/10
2645/2645 [==============================] - 753s 285ms/step - loss: 0.5536 - accuracy: 0.8585 - val_loss: 0.5991 - val_accuracy: 0.8469
Epoch 3/10
2645/2645 [==============================] - 789s 298ms/step - loss: 0.3304 - accuracy: 0.9147 - val_loss: 0.5794 - val_accuracy: 0.8526
Epoch 4/10
1869/2645 [====================>.........] - ETA: 3:36 - loss: 0.1927 - accuracy: 0.9522

In [ ]:

# If you wanted to, you could really turn this into a helper function to load in with a helper.py script...
import matplotlib.pyplot as plt

# Plot the validation and training data separately
def plot_loss_curves(history):
  """
  Returns separate loss curves for training and validation metrics.
  """
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();

In [ ]:
plot_loss_curves(mobile_history)

In [ ]:
print("Test images:")
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=IMAGE_SHAPE,
                                             batch_size=BATCH_SIZE,
                                             class_mode="categorical")

In [ ]:
mobilenet_model.evaluate(test_data)

In [ ]:
#efficient_model=create_model(effinet,num_classes=len(os.listdir('/content/bird_species_data/train')))

In [ ]:
#efficient_model.summary()

In [ ]:
'''efficient_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                      optimizer=tf.keras.optimizers.Adam(),
                      metrics=['accuracy'])
                      '''

In [ ]:
'''effinet_history=efficient_model.fit(train_data,
                                   epochs=10,
                                   steps_per_epoch=(len(train_data)),
                                   validation_data=valid_data,
                                   validation_steps=len(valid_data),
                                   callbacks=[create_tensorboard_callback(dir_name='tensorflow_hub',
                                                                          experiment_name='effinet')]
                                   )
                                   '''

In [ ]:
#plot_loss_curves(effinet_history)

In [ ]:
#efficient_model.evaluate(test_data)

# Cnn model

In [ ]:
from tensorflow.keras import layers


In [ ]:
# Define data generators for training, validation, and test sets
train_datagen = ImageDataGenerator(rescale=1/255.)
valid_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Training images:")
train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=IMAGE_SHAPE,
                                               batch_size=BATCH_SIZE,
                                               class_mode="categorical")

print("Validation images:")
valid_data = valid_datagen.flow_from_directory(valid_dir,
                                               target_size=IMAGE_SHAPE,
                                               batch_size=BATCH_SIZE,
                                               class_mode="categorical")

print("Test images:")
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=IMAGE_SHAPE,
                                             batch_size=BATCH_SIZE,
                                             class_mode="categorical")


In [ ]:
'''
def create_bird_detector(input_shape, num_classes):
    model = tf.keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

'''

In [ ]:
'''input_shape = (160, 160, 3)
num_classes = len(train_data.class_indices)

modelcnn = create_bird_detector(input_shape, num_classes)
modelcnn.summary()


modelcnn.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                 optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                 metrics=['accuracy']
)
'''

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
def create_improved_bird_detector(input_shape, num_classes):
    model = Sequential([
        layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(256, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(512, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Flatten(),
        layers.Dense(1024, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])

    return model


In [ ]:
input_shape = (160, 160, 3)
num_classes = 525

model = create_improved_bird_detector(input_shape, num_classes)

In [ ]:
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Validation and Test Data Generators
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_dir = '/content/bird_species_data/train'
valid_dir = '/content/bird_species_data/valid'
test_dir = '/content/bird_species_data/test'

In [ ]:
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(160, 160),
        batch_size=32,
        class_mode='categorical')

# Flow validation images in batches using valid_datagen generator
validation_generator = valid_datagen.flow_from_directory(
        valid_dir,
        target_size=(160, 160),
        batch_size=32,
        class_mode='categorical')

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size,
      epochs=10,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)